In [3]:
"""
ADVANCED TODO:
- write cross-validation from scratch using sklearn methods
"""

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

def sklearn_KFolding(clf, X, y, cv):

    skfolds = StratifiedKFold(cv)

    for train_index, test_index in skfolds.split(X, y):
        clone_clf = clone(clf)
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]

        clone_clf.fit(X_train, y_train)
        y_pred = clone_clf.predict(X_test)
        n_correct = sum(y_pred == y_test)
        print(n_correct / len(y_pred))